In [1115]:
% mkdir filing_texts
% mkdir prices 

mkdir: filing_texts: File exists
mkdir: prices: File exists


In [1116]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
import glob
import os 
import pandas as pd
import datetime
from scipy.stats import iqr
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import math

spm = None
english_words = None
excepted_stock_names = []
limit = -1

In [1117]:
import keras
from keras.layers import Dense, Dropout, RepeatVector, BatchNormalization, Convolution1D, Flatten, Lambda, Permute, MaxPooling1D, AlphaDropout
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.models import load_model
from tqdm import *

In [1118]:
class DateRange():
    def __init__(self, start_int, end_int):
        self.start_int = start_int
        self.end_int = end_int
        self.range = (range(start_int, end_int))

    def transform(self, date):
        output = {}
        dates = []
        for delta in self.range:
            date_delta = datetime.timedelta(days=delta)
            date_string = datetime.datetime.strptime(date, '%Y-%m-%d').date()
            dates.append(str(date_string + date_delta))
        return dates

In [1119]:
class SNDM():
    def __init__(self, limit=-1):
        self.limit = limit
        self.sndm = self.__get_sndm()

    def __get_sndm(self):
        spm = self.__stock_name_date_mapping()
        output = {}
        for stock_name in spm.keys():
            numerical_prices = 0
            for date in DateRange(-3,2).transform(spm[stock_name]['date']):
                try:
                    isnan = math.isnan(PriceData(stock_name).on_date(date, 'open'))
                except:
                    isnan = True
                if not (isnan):
                    numerical_prices += 1 
            # Delete stock_name from spm if numerical prices under a threshold. (nans)
            if numerical_prices >= 3:
                output[stock_name] = spm[stock_name]
        return output
    
    def __stock_name_date_mapping(self):
        spm = {}
        for i, file in enumerate(glob.glob('filing_texts/*')[0:self.limit]):
            try:
                st_name = self.__stock_name_from_filename(file)
                file_data = open(file, 'r').read()[:200]
                time_data = file_data.split('<ACCEPTANCE-DATETIME>')[1].split('\\n')[0][:8]
                year = time_data[0:4]
                month = time_data[4:6]
                day = time_data[6:8]
                date_stamp = ("%s-%s-%s" %(year, month, day))
                spm[st_name] = {'date': date_stamp}
            except:
                print("No data for ", file)
                continue
        return spm
    def __stock_name_from_filename(self, filename):
        return filename.split('/')[-1].split('_')[0]

In [1120]:
spm = SNDM().sndm

In [1121]:
class PriceData:
    def __init__(self, stock_name):
        self.stock_name = stock_name
        try:
            self.price_data = pd.read_csv('prices/' + stock_name + '.csv')
        except:
            self.price_data = pd.DataFrame.from_dict({})

    def on_date(self, date, market_time = 'open'): 
        try:
            return float(self.price_data.loc[self.price_data['date'] == date][market_time])
        except: 
            return None

In [1122]:
class FilenamesToStockNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        output = []
        for filename in X:
            stock_name = self.__stock_name_from_filename(filename)
            output.append(stock_name)
        return output
    def __stock_name_from_filename(self, filename):
        return filename.split('/')[-1].split('_')[0]

In [1123]:
## Used in y pipeline
class SpmToFileNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
    def transform(self, X):
        output = []
        for key in X.keys():
            date = X[key]['date']
            output.append(f'filing_texts/{key}_{date}')
        return output
    
class SpmToStockNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return X.keys()
    
class StockNamesToFileNamesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
    
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return [f'filing_texts/{stock_name}' for stock_name in X]
    
# class MapStockNamesToDatesTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, start_int, end_int):
#         self.start_int = start_int
#         self.end_int = end_int
#         self.range = (range(start_int, end_int))
    
#     def fit(self, X, y=None):
#         return self
    
#     def fit_transform(self, X, y=None):
#         return self.fit(X, y).transform(X)
        
#     def transform(self, X):
#         output = {}
#         for i, stock_name in enumerate(X):
#             try:
#                 date = spm[stock_name]['date']
#             except:
#                 continue # Ignore stocks which don't have a date
#             dates = []
#             for delta in self.range:
#                 date_delta = datetime.timedelta(days=delta)
#                 date_string = datetime.datetime.strptime(date, '%Y-%m-%d').date()
#                 dates.append(str(date_string + date_delta))
#             output[stock_name] = dates
#         return output
# class StockNameDatesMapToPricesListTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         return None
    
#     def fit(self, X, y=None):
#         return self
    
#     def fit_transform(self, X, y=None):
#         return self.fit(X, y).transform(X)
        
#     def transform(self, X):
#         output = []
#         for stock_name in X.keys():
#             prices = []
#             for date in X[stock_name]:
#                 prices.append(PriceData(stock_name).on_date(date, 'open'))
#             output.append(prices)
#         return np.array(output)
    
# class LabelsTransform(BaseEstimator, TransformerMixin):
#     # Returns the interquartile-range and median.
#     def __init__(self):
#         return None
        
#     def fit(self, X, y=None):
#         return self
    
#     def fit_transform(self, X, y=None):
#         return self.fit(X, y).transform(X)
        
#     def transform(self, X):
#         # ldcom = last_day_change_over_median
#         print(X.shape)
#         ldcom = []
#         for prices in X:
#             this_median = np.median(prices[0:-3])
#             ldcom.append(((prices[-1]-this_median)/this_median))
#         return np.array(ldcom)
            
class StatisticalMeasuresTransformer(BaseEstimator, TransformerMixin):
    # Returns the interquartile-range and median.
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        # ldcom = last_day_change_over_median
        
        output = []
        self.iqr_var = []
        self.median = []
        for prices in X:
            this_iqr = iqr(prices[0:-3])
            this_median = np.median(prices[0:-3])
            self.iqr_var.append(this_iqr)
            self.median.append(this_median)
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        X_output = []
        for i, prices in enumerate(X):
            stats   = []
            iqr_var = self.iqr_var[i] or iqr(prices)
            median  = self.median[i]  or np.median(prices)
            
            stats.append(iqr_var)
            stats.append(median)
            
            X_output.append(stats)
        return np.array(X_output)
class SparseToArray(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return np.array(X.toarray()) #[ar.toarray() for ar in X]
    
class ReadFiles(BaseEstimator, TransformerMixin): 
    def __init__(self):
        return None
        
    def fit(self, X, y=None):
        return self
    
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        return (open(filename, 'r').read() for filename in tqdm(X))
            

In [1124]:
def get_filenames(limit = -1):
    filenames = []
    directory_files = glob.glob('filing_texts/*')
    excepted_files = [('filing_texts/' + sn) for sn in excepted_stock_names]
    filenames_with_data = [x for x in directory_files[:limit] if x not in excepted_files]
    for filename in filenames_with_data:
        filenames.append(filename)
    return filenames

In [1125]:
class StockNamesToLabelsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, start_int, end_int):
        self.filing_int = start_int * -1
        self.start_int = start_int
        self.end_int = end_int
        self.range = (range(start_int, end_int))
    # Not implemented since only used to generate labels
    def fit(self, X, y=None):
        return self
    # Not implemented since only used to generate labels
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
        
    def transform(self, X):
        h = {}
        for i, stock_name in enumerate(X):
            try:
                date = spm[stock_name]['date']
            except:
                continue # Ignore stocks which don't have a date
            dates = []
            for delta in self.range:
                date_delta = datetime.timedelta(days=delta)
                date_string = datetime.datetime.strptime(date, '%Y-%m-%d').date()
                dates.append(str(date_string + date_delta))
            h[stock_name] = dates
        
        ldcom = []
        for stock_name in h.keys():
            earliest_price_after_filing = None
            hist_p = []
            for i, date in enumerate(h[stock_name]):
                price = PriceData(stock_name).on_date(date, 'open')
                if price and not math.isnan(price):
                    if i > self.filing_int:
                        earliest_price_after_filing = earliest_price_after_filing or price
                    else:
                        hist_p.append(price)
            # Closing price on day of filing
            price_close_filing = PriceData(stock_name).on_date(h[stock_name][self.filing_int], 'close')
            # Use either the next open day of trading or the close price on day of filing
            comparison_price = earliest_price_after_filing or price_close_filing
            # Remove nans from historical prices before taking the mean
            this_mean = np.mean(hist_p)
            ldcom.append(((comparison_price-this_mean)/this_mean))
        return np.array(ldcom)

In [1126]:
StockNamesToLabelsTransformer(-5,5).transform(['XOXO'])

array([ 0.01682823])

In [1127]:
# Used to build other pipelines
from sklearn.preprocessing import Imputer

# prices_pipeline = Pipeline([
#     ('spm_to_filenames', SpmToFileNamesTransformer()),
#     ('filenames_to_stock_names', FilenamesToStockNamesTransformer()),
#     ('stock_names_to_dates', MapStockNamesToDatesTransformer(-5, 2)),
#     ('dates_to_prices_transformer', StockNameDatesMapToPricesListTransformer()),
#     ('imputer', Imputer(axis=1))
# ])

labels_pipeline = Pipeline([
    ('spm_to_filenames', SpmToFileNamesTransformer()),
    ('filenames_to_stock_names', FilenamesToStockNamesTransformer()),
    ('dates_to_prices_transformer', StockNamesToLabelsTransformer(-5,2))
])

# Used as the final y values 
# labels_pipeline = Pipeline([
#     ('prices_pipeline', prices_pipeline),
#     ('labels_transform', LabelsTransform())
# ])

# Used for training and test set features
stock_stats = Pipeline([
    ('prices_pipeline', prices_pipeline),
    ('stats_transform', StatisticalMeasuresTransformer())
])

# Used for training and test set features 
# ('stock_names_to_file_names', StockNamesToFileNamesTransformer()),
text_word_counts = Pipeline([
    ('spm_to_file_names', SpmToFileNamesTransformer()),
    ('read_files', ReadFiles()),
    ('vect', TfidfVectorizer(
                token_pattern=r"[a-zA-Z]+", 
                min_df = 0.10,
                max_df = 0.80,
                stop_words = 'english',
                max_features=9000,
                ngram_range=(1, 1))),
    ('sparse_to_array', SparseToArray()),
    ('std_scaler', StandardScaler()),
])


In [1128]:
X = text_word_counts.fit_transform(spm)
y = labels_pipeline.fit_transform(spm)
print(X.shape)
print(y.shape)






  0%|          | 0/188 [00:00<?, ?it/s]




  1%|          | 2/188 [00:00<00:20,  9.14it/s]




  5%|▌         | 10/188 [00:00<00:12, 14.09it/s]




  7%|▋         | 14/188 [00:00<00:10, 16.44it/s]




  9%|▊         | 16/188 [00:02<00:22,  7.68it/s]




 13%|█▎        | 24/188 [00:02<00:15, 10.83it/s]




 14%|█▍        | 27/188 [00:02<00:14, 11.12it/s]




 18%|█▊        | 34/188 [00:02<00:13, 11.59it/s]




 21%|██        | 39/188 [00:03<00:11, 12.72it/s]




 22%|██▏       | 42/188 [00:03<00:11, 12.80it/s]




 24%|██▍       | 46/188 [00:03<00:10, 13.60it/s]




 30%|██▉       | 56/188 [00:03<00:08, 16.01it/s]




 32%|███▏      | 61/188 [00:03<00:07, 16.82it/s]




 35%|███▌      | 66/188 [00:04<00:07, 16.44it/s]




 37%|███▋      | 70/188 [00:06<00:11, 10.67it/s]




 42%|████▏     | 79/188 [00:06<00:09, 11.86it/s]




 45%|████▍     | 84/188 [00:06<00:08, 12.12it/s]




 47%|████▋     | 88/188 [00:07<00:08, 11.80it/s]




 50%|█████     | 94/188 [00:07<00:07, 12.41it/s]




(188, 6004)
(188,)


In [1129]:
X_train, X_test, y_train_continuous, y_test_continuous = train_test_split(X, y, test_size=0.2, random_state=42)

In [1130]:
def bool_arr(arr, limit=0.5):
    y_bool = []
    for num in arr:
        if num >= limit:
            y_bool.append(1)
        else:
            y_bool.append(0)
    return np.array(y_bool)
y_train = bool_arr(y_train_continuous, 0.05)
y_test = bool_arr(y_test_continuous, 0.00)

In [1131]:
from sklearn.feature_selection import SelectFpr
from sklearn.model_selection import GridSearchCV, cross_val_score

svc_train_pipeline = Pipeline([
    ('reduce_false_pos', SelectFpr(alpha=0.1)),
    ('svc', GridSearchCV(estimator=SVC(gamma='auto'), param_grid=dict(C=np.logspace(-1,3)), n_jobs=1))
])
svc_train_pipeline.fit(X_train, y_train)
svc_train_pipeline.score(X_test, y_test)

0.44736842105263158

In [1132]:
from sklearn.svm import SVC

clf = SVC(C=100)
# clf = RandomForestRegressor(n_estimators=200, max_depth=3, verbose=1, n_jobs=2)
clf.fit(X_train, y_train)

/Users/Piper/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [1133]:
from keras import regularizers
from sklearn.metrics import confusion_matrix

In [1151]:
class DeepEstimator():
    def __init__(self):
        return None
    def fit(self, X, y, X_val, y_val):
        class_weight = {0: 0.1,
                1: 0.9}
        self.model = self.__define_model(X)
        sched = [[0.0001, 2], [0.001, 20], [0.01, 2], [0.1, 2], [0.5, 1], [0.1, 5], [0.01, 20], [0.001, 40], [0.0001, 80], [0.00005, 120]]
        for lr, epochs in sched:
            self.model.optimizer.lr = lr
            self.model.fit(np.array(X), np.array(y), epochs=epochs,  class_weight=class_weight, batch_size=64, validation_data=(X_val, y_val))
        return self
    def predict(self, X, y=None):
        return self.model.predict(X)
    def __define_model(self, X):
        shape = X.shape[1]
        model = Sequential([
            BatchNormalization(input_shape=(shape,)),
            Dense(25, activation='relu', kernel_regularizer=regularizers.l1(0.01)),
            Dropout(0.8),
            BatchNormalization(),
            Dense(1, activation='sigmoid')   
        ])

        model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        return model
    

In [1157]:
X_train, X_test, y_train_continuous, y_test_continuous = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = bool_arr(y_train_continuous, 0.05)
y_test = bool_arr(y_test_continuous, 0.00)

select_fpr = SelectFpr(alpha=0.1)

X_train = select_fpr.fit_transform(X_train, y_train)
X_test = select_fpr.transform(X_test)

DeepEstimator().fit(X_train, y_train, X_test, y_test)



Train on 150 samples, validate on 38 samples
Epoch 1/2
150/150 [==============================] - 3s 20ms/step - loss: 9.3447 - acc: 0.5467 - val_loss: 9.8915 - val_acc: 0.5526
Epoch 2/2
150/150 [==============================] - 0s 190us/step - loss: 9.2921 - acc: 0.5867 - val_loss: 9.8204 - val_acc: 0.5526
Train on 150 samples, validate on 38 samples
Epoch 1/20
150/150 [==============================] - 0s 185us/step - loss: 9.2166 - acc: 0.5533 - val_loss: 9.7557 - val_acc: 0.5526
Epoch 2/20
150/150 [==============================] - 0s 265us/step - loss: 9.1698 - acc: 0.5200 - val_loss: 9.6970 - val_acc: 0.5526
Epoch 3/20
150/150 [==============================] - 0s 203us/step - loss: 9.0677 - acc: 0.6333 - val_loss: 9.6341 - val_acc: 0.5526
Epoch 4/20
150/150 [==============================] - 0s 217us/step - loss: 9.0662 - acc: 0.5733 - val_loss: 9.5749 - val_acc: 0.5526
Epoch 5/20
150/150 [==============================] - 0s 157us/step - loss: 8.9575 - acc: 0.5800 - val_loss: 

150/150 [==============================] - 0s 275us/step - loss: 6.2212 - acc: 0.6333 - val_loss: 6.7950 - val_acc: 0.5000
Epoch 8/40
150/150 [==============================] - 0s 347us/step - loss: 6.1811 - acc: 0.5467 - val_loss: 6.7477 - val_acc: 0.5000
Epoch 9/40
150/150 [==============================] - 0s 357us/step - loss: 6.1381 - acc: 0.6733 - val_loss: 6.7011 - val_acc: 0.5000
Epoch 10/40
150/150 [==============================] - 0s 249us/step - loss: 6.0795 - acc: 0.6067 - val_loss: 6.6563 - val_acc: 0.5000
Epoch 11/40
150/150 [==============================] - 0s 387us/step - loss: 6.0234 - acc: 0.6733 - val_loss: 6.6088 - val_acc: 0.5000
Epoch 12/40
150/150 [==============================] - 0s 298us/step - loss: 5.9916 - acc: 0.6000 - val_loss: 6.5645 - val_acc: 0.5000
Epoch 13/40
150/150 [==============================] - 0s 347us/step - loss: 5.9533 - acc: 0.6000 - val_loss: 6.5214 - val_acc: 0.5000
Epoch 14/40
150/150 [==============================] - 0s 358us/step 

Epoch 27/80
150/150 [==============================] - 0s 350us/step - loss: 3.7872 - acc: 0.7133 - val_loss: 4.4466 - val_acc: 0.5000
Epoch 28/80
150/150 [==============================] - 0s 452us/step - loss: 3.7513 - acc: 0.6600 - val_loss: 4.4139 - val_acc: 0.5000
Epoch 29/80
150/150 [==============================] - 0s 295us/step - loss: 3.7323 - acc: 0.6333 - val_loss: 4.3807 - val_acc: 0.5000
Epoch 30/80
150/150 [==============================] - 0s 308us/step - loss: 3.6945 - acc: 0.6533 - val_loss: 4.3487 - val_acc: 0.5000
Epoch 31/80
150/150 [==============================] - 0s 348us/step - loss: 3.6654 - acc: 0.6800 - val_loss: 4.3133 - val_acc: 0.5000
Epoch 32/80
150/150 [==============================] - 0s 263us/step - loss: 3.6176 - acc: 0.6667 - val_loss: 4.2788 - val_acc: 0.5000
Epoch 33/80
150/150 [==============================] - 0s 381us/step - loss: 3.5841 - acc: 0.6467 - val_loss: 4.2475 - val_acc: 0.5000
Epoch 34/80
150/150 [==============================] - 

150/150 [==============================] - 0s 201us/step - loss: 2.0555 - acc: 0.7333 - val_loss: 2.8428 - val_acc: 0.4737
Epoch 8/120
150/150 [==============================] - 0s 245us/step - loss: 2.0477 - acc: 0.7667 - val_loss: 2.8184 - val_acc: 0.4737
Epoch 9/120
150/150 [==============================] - 0s 281us/step - loss: 2.0263 - acc: 0.7533 - val_loss: 2.7931 - val_acc: 0.4737
Epoch 10/120
150/150 [==============================] - 0s 231us/step - loss: 1.9877 - acc: 0.7733 - val_loss: 2.7737 - val_acc: 0.4737
Epoch 11/120
150/150 [==============================] - ETA: 0s - loss: 1.9751 - acc: 0.703 - 0s 297us/step - loss: 1.9661 - acc: 0.7200 - val_loss: 2.7608 - val_acc: 0.4737
Epoch 12/120
150/150 [==============================] - 0s 324us/step - loss: 1.9296 - acc: 0.7867 - val_loss: 2.7491 - val_acc: 0.4737
Epoch 13/120
150/150 [==============================] - 0s 245us/step - loss: 1.9250 - acc: 0.7667 - val_loss: 2.7254 - val_acc: 0.4737
Epoch 14/120
150/150 [===

150/150 [==============================] - 0s 405us/step - loss: 1.0232 - acc: 0.8133 - val_loss: 1.8576 - val_acc: 0.5000
Epoch 68/120
150/150 [==============================] - 0s 345us/step - loss: 1.0267 - acc: 0.7467 - val_loss: 1.8423 - val_acc: 0.5000
Epoch 69/120
150/150 [==============================] - 0s 448us/step - loss: 1.0129 - acc: 0.8067 - val_loss: 1.8349 - val_acc: 0.5000
Epoch 70/120
150/150 [==============================] - 0s 280us/step - loss: 0.9898 - acc: 0.8133 - val_loss: 1.8291 - val_acc: 0.5000
Epoch 71/120
150/150 [==============================] - 0s 292us/step - loss: 0.9843 - acc: 0.8067 - val_loss: 1.8187 - val_acc: 0.5000
Epoch 72/120
150/150 [==============================] - 0s 340us/step - loss: 0.9895 - acc: 0.8067 - val_loss: 1.7957 - val_acc: 0.5000
Epoch 73/120
150/150 [==============================] - ETA: 0s - loss: 0.9802 - acc: 0.781 - 0s 337us/step - loss: 0.9727 - acc: 0.7800 - val_loss: 1.7814 - val_acc: 0.5000
Epoch 74/120
150/150 [=

In [1161]:
X_test.shape

(38, 924)

In [1159]:
deep_train_pipeline = Pipeline([
    ('reduce_false_pos', select_fpr),
    ('deep', DeepEstimator())
])

In [1162]:
score(deep_train_pipeline, X_test, y_test_continuous,0.5)

ValueError: X has a different shape than during fitting.

In [1160]:
import matplotlib.pyplot as plt
%matplotlib inline

def score(est, X, y, cutoff=0.75):
    y_pred = est.predict(X)
    # Invest if over 50% confident 
    y_pred_bool = bool_arr(y_pred, cutoff)
    # Score correct if you make postive returns
    y_true_bool = bool_arr(y, 0)
    total = np.dot(y_pred_bool.reshape(X.shape[0],), y)
    buy_all = np.dot(np.ones(X.shape[0]), y)
    av = total/len(X)
    buy_all_av = buy_all/len(X)
#     print(confusion_matrix(y_true_bool, y_pred_bool))
    return [av, buy_all_av]


ys_attained = []
ys_potential = []
xs = []
for i in np.linspace(0, 1):
    sc = score(deep_train_pipeline, X_test, y_test_continuous,i)
    ys_attained.append(sc[0])
    ys_potential.append(sc[1])
    xs.append(i)
plt.plot(xs, ys_attained)
plt.plot(xs, ys_potential)


ValueError: X has a different shape than during fitting.

In [ ]:
bool_arr(deep_train_pipeline.predict(X_test))

In [ ]:
# # Factors to look into including: 
# "BookValue" = (Total Assets - Total Liabilities) / Number of shares outstanding
# "MarketCap" = Market price per share * number of shares 
# "DividendYield" = Dividend / Market price per share 
# "EarningsPerShare" 
# "PERatio2" = Market price per share / earning per share 
# "priceBook" = Market price per share / ((Total Assets - Total Liabilities) / Number of shares outstanding)
# "PriceSales" = MarketCap / Revenue 
# "Ask"


In [ ]:
# vocab = list(text_word_counts.steps[2][1].vocabulary_.keys())
# import operator
# iv_dict = [[vocab[i],-float(f)] for i,f in enumerate(clf.coef_)]
# most_important_terms = sorted(iv_dict, key=operator.itemgetter(1))[0:100]
# print(most_important_terms)
# most_important_vocab = dict(most_important_terms).keys()

In [ ]:
# from joblib import Memory
# %mkdir cachedir
# location = './cachedir'
# memory = Memory(location, verbose=0)
# stock_name_date_mapping = memory.cache(stock_name_date_mapping)

In [ ]:
# # print(text_word_counts.steps[1][1].vocabulary_)
# def get_excepted_stock_names(limit=-1):
#     esn = {}
#     for i, file in tqdm(enumerate(glob.glob('filing_texts/*')[0:limit])):
#         stock_name = stock_name_from_filename(file)
#         file_data = open(file, 'r').read()
#         try:
#             with open(file) as open_file:
#                 file_data = [next(open_file) for x in range(3)]
#             file_data = ''.join(file_data)
#             time_data = file_data.split("\n")[2][5:14]
#         except:
#             esn[i] = stock_name
#             continue
#     return esn

# def get_x_mask(X, y, filenames):
#     no_date_indices = []
#     if X.shape[0] != len(y):
#         stock_names = FilenamesToStockNamesTransformer().transform(filenames)
#         for i, stock_name in enumerate(stock_names): 
#             try:
#                 date = spm[stock_name]['date']
#             except:
#                 no_date_indices.append(i)
#         mask = np.ones(X.shape[0], dtype=bool)
#         mask[no_date_indices] = False
#         X_mask = X[mask]
#     else:
#         X_mask = X
#     return X_mask